In [ ]:
import sys
import os
import math
import pandas as pd
from PySide6.QtWidgets import (
    QApplication, QMainWindow, QVBoxLayout, QWidget, QPushButton,
    QLabel, QFileDialog, QMessageBox, QHBoxLayout, QTextEdit,
    QDialog, QLineEdit, QFrame, QGroupBox, QProgressDialog, QSlider
)
from PySide6.QtCore import Qt, QSize
from PySide6.QtGui import (QPixmap, QIcon, QFont, QPalette, QColor, 
                          QDoubleValidator, QLinearGradient)
from ultralytics import YOLO

class FuturisticButton(QPushButton):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setMinimumHeight(45)
        self.setCursor(Qt.PointingHandCursor)
        self.setStyleSheet("""
            QPushButton {
                background-color: #00a8ff;
                color: white;
                font-weight: bold;
                border: none;
                border-radius: 8px;
                padding: 12px 20px;
                font-size: 16px;
                min-width: 120px;
            }
            QPushButton:hover {
                background-color: #0097e6;
            }
            QPushButton:pressed {
                background-color: #0078c4;
            }
        """)

class ModelTrainingWindow(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Обучение модели")
        self.setMinimumSize(700, 550)
        self.setStyleSheet("""
            QDialog {
                background: qlineargradient(x1:0, y1:0, x2:1, y2:1,
                    stop:0 #1a1a2e, stop:1 #16213e);
            }
            QGroupBox {
                border: 2px solid #00a8ff;
                border-radius: 8px;
                margin-top: 15px;
                padding-top: 20px;
                font-weight: bold;
                color: #00a8ff;
                font-size: 16px;
                background-color: rgba(26, 26, 46, 150);
            }
            QLabel {
                color: #e0e0e0;
                font-size: 14px;
            }
            QLineEdit {
                background-color: rgba(255, 255, 255, 20);
                border: 1px solid #00a8ff;
                border-radius: 6px;
                padding: 10px;
                font-size: 14px;
                color: white;
                selection-background-color: #00a8ff;
            }
            QLineEdit:focus {
                border: 2px solid #00a8ff;
            }
        """)
        self.initUI()
        self.setWindowIcon(QIcon("icons/train.png"))

    def initUI(self):
        layout = QVBoxLayout()
        layout.setContentsMargins(25, 25, 25, 25)
        layout.setSpacing(20)

        model_group = QGroupBox("Настройки модели")
        model_layout = QVBoxLayout()
        model_layout.setSpacing(15)

        self.model_name_label = QLabel("Название модели:")
        self.model_name_input = QLineEdit()
        self.model_name_input.setPlaceholderText("Введите название модели")

        self.model_version_label = QLabel("Версия модели (s, m, l):")
        self.model_version_input = QLineEdit()
        self.model_version_input.setPlaceholderText("Введите s, m, или l")

        model_layout.addWidget(self.model_name_label)
        model_layout.addWidget(self.model_name_input)
        model_layout.addWidget(self.model_version_label)
        model_layout.addWidget(self.model_version_input)
        model_group.setLayout(model_layout)

        train_group = QGroupBox("Настройки обучения")
        train_layout = QVBoxLayout()
        train_layout.setSpacing(15)

        self.epochs_label = QLabel("Количество эпох:")
        self.epochs_input = QLineEdit()
        self.epochs_input.setPlaceholderText("Введите количество эпох")
        self.epochs_input.setValidator(QDoubleValidator(1, 1000, 0))

        train_layout.addWidget(self.epochs_label)
        train_layout.addWidget(self.epochs_input)
        train_group.setLayout(train_layout)

        data_group = QGroupBox("Данные")
        data_layout = QVBoxLayout()
        data_layout.setSpacing(15)

        self.data_button = FuturisticButton("Выбрать данные")
        self.data_button.setIcon(QIcon("icons/folder.png"))
        self.data_button.clicked.connect(self.select_data)
        self.data_label = QLabel("Данные не выбраны")
        self.data_label.setWordWrap(True)
        self.data_label.setStyleSheet("color: #00a8ff;")

        self.save_path_button = FuturisticButton("Выбрать путь")
        self.save_path_button.setIcon(QIcon("icons/save.png"))
        self.save_path_button.clicked.connect(self.select_save_path)
        self.save_path_label = QLabel("Путь не выбран")
        self.save_path_label.setWordWrap(True)
        self.save_path_label.setStyleSheet("color: #00a8ff;")

        data_layout.addWidget(self.data_button)
        data_layout.addWidget(self.data_label)
        data_layout.addWidget(self.save_path_button)
        data_layout.addWidget(self.save_path_label)
        data_group.setLayout(data_layout)

        layout.addWidget(model_group)
        layout.addWidget(train_group)
        layout.addWidget(data_group)

        self.train_button = FuturisticButton("Начать обучение")
        self.train_button.setIcon(QIcon("icons/start.png"))
        self.train_button.clicked.connect(self.start_training)
        layout.addWidget(self.train_button)

        self.setLayout(layout)

    def select_data(self):
        data_path = QFileDialog.getExistingDirectory(self, "Выберите папку с данными")
        if data_path:
            self.data_path = data_path
            self.data_label.setText(f"Выбраны данные: {os.path.basename(data_path)}")
            self.data_label.setStyleSheet("color: #00a8ff;")

            data_yaml_path = os.path.join(data_path, "data.yaml")
            if not os.path.exists(data_yaml_path):
                self.data_label.setText(f"Выбраны данные: {os.path.basename(data_path)}\n(файл data.yaml не найден!)")
                self.data_label.setStyleSheet("color: #ff4757;")

    def select_save_path(self):
        save_path = QFileDialog.getExistingDirectory(self, "Выберите папку для сохранения модели")
        if save_path:
            self.save_path = save_path
            self.save_path_label.setText(f"Модель будет сохранена в: {save_path}")
            self.save_path_label.setStyleSheet("color: #00a8ff;")

    def start_training(self):
        model_name = self.model_name_input.text().strip()
        model_version = self.model_version_input.text().strip().lower()
        epochs = self.epochs_input.text().strip()
        data_path = getattr(self, 'data_path', None)
        save_path = getattr(self, 'save_path', None)

        if not model_name:
            self.show_error("Введите название модели!")
            return
        if model_version not in ['s', 'm', 'l']:
            self.show_error("Версия модели должна быть s, m, или l!")
            return
        if not epochs.isdigit() or int(epochs) <= 0:
            self.show_error("Количество эпох должно быть положительным числом!")
            return
        if not data_path:
            self.show_error("Выберите данные для обучения!")
            return
        if not save_path:
            self.show_error("Выберите путь для сохранения модели!")
            return

        data_yaml_path = os.path.join(data_path, "data.yaml")
        if not os.path.exists(data_yaml_path):
            self.show_error(f"Файл data.yaml не найден в папке с данными!\nОжидаемый путь: {data_yaml_path}")
            return

        model_path = f"yolov8{model_version}.pt"

        try:
            self.train_button.setEnabled(False)
            self.train_button.setText("Обучение...")
            QApplication.processEvents()

            model = YOLO(model_path)
            results = model.train(
                data=data_yaml_path,
                epochs=int(epochs),
                project=save_path,
                name=model_name,
            )

            self.show_success("Модель успешно обучена и сохранена!")
            self.close()

        except Exception as e:
            self.show_error(f"Произошла ошибка при обучении: {str(e)}")
        finally:
            self.train_button.setEnabled(True)
            self.train_button.setText("Начать обучение")

    def show_error(self, message):
        msg = QMessageBox(self)
        msg.setIcon(QMessageBox.Critical)
        msg.setWindowTitle("Ошибка")
        msg.setText(message)
        msg.setStyleSheet("""
            QMessageBox {
                background-color: #1a1a2e;
            }
            QLabel {
                color: white;
            }
        """)
        msg.exec()

    def show_success(self, message):
        msg = QMessageBox(self)
        msg.setIcon(QMessageBox.Information)
        msg.setWindowTitle("Успех")
        msg.setText(message)
        msg.setStyleSheet("""
            QMessageBox {
                background-color: #1a1a2e;
            }
            QLabel {
                color: white;
            }
        """)
        msg.exec()

class ResultsWindow(QDialog):
    def __init__(self, result_image_paths, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Результаты анализа")
        self.setMinimumSize(800, 600)
        self.setStyleSheet("""
            QDialog {
                background: qlineargradient(x1:0, y1:0, x2:1, y2:1,
                    stop:0 #1a1a2e, stop:1 #16213e);
            }
            QLabel {
                color: #e0e0e0;
            }
        """)
        self.result_image_paths = result_image_paths
        self.current_index = 0
        self.initUI()
        self.setWindowIcon(QIcon("icons/results.png"))

    def initUI(self):
        layout = QVBoxLayout()
        layout.setContentsMargins(20, 20, 20, 20)
        layout.setSpacing(15)

        self.image_frame = QFrame()
        self.image_frame.setFrameShape(QFrame.StyledPanel)
        self.image_frame.setStyleSheet("""
            QFrame {
                background-color: rgba(40, 40, 60, 200);
                border-radius: 8px;
            }
        """)
        frame_layout = QVBoxLayout()
        frame_layout.setContentsMargins(10, 10, 10, 10)

        self.image_label = QLabel()
        self.image_label.setAlignment(Qt.AlignCenter)
        frame_layout.addWidget(self.image_label)

        self.image_frame.setLayout(frame_layout)
        layout.addWidget(self.image_frame)

        nav_frame = QFrame()
        nav_layout = QHBoxLayout()
        nav_layout.setContentsMargins(0, 0, 0, 0)

        self.prev_button = FuturisticButton("")
        self.prev_button.setIcon(QIcon("icons/arrow-left.png"))
        self.prev_button.setIconSize(QSize(24, 24))
        self.prev_button.setFixedSize(40, 40)
        self.prev_button.clicked.connect(self.show_previous_image)

        self.next_button = FuturisticButton("")
        self.next_button.setIcon(QIcon("icons/arrow-right.png"))
        self.next_button.setIconSize(QSize(24, 24))
        self.next_button.setFixedSize(40, 40)
        self.next_button.clicked.connect(self.show_next_image)

        self.result_label = QLabel()
        self.result_label.setAlignment(Qt.AlignCenter)
        self.result_label.setStyleSheet("font-weight: bold; color: #00a8ff;")

        nav_layout.addStretch()
        nav_layout.addWidget(self.prev_button)
        nav_layout.addWidget(self.result_label)
        nav_layout.addWidget(self.next_button)
        nav_layout.addStretch()

        nav_frame.setLayout(nav_layout)
        layout.addWidget(nav_frame)

        self.setLayout(layout)
        self.show_image(self.current_index)

    def show_image(self, index):
        if 0 <= index < len(self.result_image_paths):
            pixmap = QPixmap(self.result_image_paths[index])
            self.image_label.setPixmap(pixmap.scaled(
                self.image_frame.width() - 40,
                self.image_frame.height() - 40,
                Qt.KeepAspectRatio,
                Qt.SmoothTransformation
            ))
            self.result_label.setText(f"Изображение {index + 1} из {len(self.result_image_paths)}")
            self.prev_button.setEnabled(index > 0)
            self.next_button.setEnabled(index < len(self.result_image_paths) - 1)

    def show_previous_image(self):
        if self.current_index > 0:
            self.current_index -= 1
            self.show_image(self.current_index)

    def show_next_image(self):
        if self.current_index < len(self.result_image_paths) - 1:
            self.current_index += 1
            self.show_image(self.current_index)

    def resizeEvent(self, event):
        self.show_image(self.current_index)
        super().resizeEvent(event)

class ImageAnalysisWindow(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Анализ изображений")
        self.setMinimumSize(500, 400)
        self.setStyleSheet("""
            QDialog {
                background: qlineargradient(x1:0, y1:0, x2:1, y2:1,
                    stop:0 #1a1a2e, stop:1 #16213e);
            }
            QGroupBox {
                border: 2px solid #00a8ff;
                border-radius: 8px;
                margin-top: 15px;
                padding-top: 20px;
                font-weight: bold;
                color: #00a8ff;
                font-size: 16px;
                background-color: rgba(26, 26, 46, 150);
            }
            QLabel {
                color: #e0e0e0;
                font-size: 14px;
            }
        """)
        self.initUI()
        self.setWindowIcon(QIcon("icons/analyze.png"))

    def initUI(self):
        layout = QVBoxLayout()
        layout.setContentsMargins(20, 20, 20, 20)
        layout.setSpacing(15)

        settings_group = QGroupBox("Настройки анализа")
        settings_layout = QVBoxLayout()
        settings_layout.setSpacing(15)

        self.threshold_label = QLabel("Порог значимости:")
        
        self.threshold_slider = QSlider(Qt.Horizontal)
        self.threshold_slider.setRange(0, 100)
        self.threshold_slider.setValue(50)
        self.threshold_slider.setTickInterval(10)
        self.threshold_slider.setTickPosition(QSlider.TicksBelow)
        self.threshold_slider.setStyleSheet("""
            QSlider::groove:horizontal {
                height: 8px;
                background: #2d3436;
                border-radius: 4px;
            }
            QSlider::handle:horizontal {
                width: 16px;
                height: 16px;
                background: #00a8ff;
                border-radius: 8px;
                margin: -4px 0;
            }
            QSlider::sub-page:horizontal {
                background: #00a8ff;
                border-radius: 4px;
            }
        """)
        
        self.threshold_value_label = QLabel("0.50")
        self.threshold_value_label.setStyleSheet("color: #00a8ff; font-weight: bold;")
        self.threshold_slider.valueChanged.connect(self.update_threshold_label)

        settings_layout.addWidget(self.threshold_label)
        settings_layout.addWidget(self.threshold_slider)
        settings_layout.addWidget(self.threshold_value_label)
        settings_group.setLayout(settings_layout)
        layout.addWidget(settings_group)

        self.load_images_button = FuturisticButton("Загрузить изображения")
        self.load_images_button.setIcon(QIcon("icons/images.png"))
        self.load_images_button.clicked.connect(self.load_images)
        layout.addWidget(self.load_images_button)

        self.setLayout(layout)

    def update_threshold_label(self, value):
        threshold = value / 100
        self.threshold_value_label.setText(f"{threshold:.2f}")

    def load_images(self):
        if not hasattr(self.parent(), 'selected_model') and not hasattr(self.parent(), 'trained_model_path'):
            self.show_error("Сначала выберите или загрузите модель!")
            return

        threshold = self.threshold_slider.value() / 100

        image_paths, _ = QFileDialog.getOpenFileNames(
            self,
            "Выберите изображения",
            "",
            "Image Files (*.jpg *.jpeg *.png *.bmp)"
        )
        if not image_paths:
            return

        if hasattr(self.parent(), 'trained_model_path'):
            model_path = self.parent().trained_model_path
        else:
            model_path = f"{self.parent().selected_model}.pt"

        model = YOLO(model_path)
        result_image_paths = []

        progress = QProgressDialog("Анализ изображений...", "Отмена", 0, len(image_paths), self)
        progress.setWindowModality(Qt.WindowModal)
        progress.setStyleSheet("""
            QProgressDialog {
                background-color: #1a1a2e;
                color: white;
            }
            QLabel {
                color: white;
            }
        """)

        for i, image_path in enumerate(image_paths):
            progress.setValue(i)
            if progress.wasCanceled():
                break

            try:
                prediction = model.predict(
                    source=image_path,
                    conf=threshold,
                    save=True,
                    exist_ok=True
                )
                if len(prediction[0].boxes) > 0:
                    result_image_path = os.path.join(prediction[0].save_dir, os.path.basename(image_path))
                    result_image_paths.append(result_image_path)
            except Exception as e:
                self.show_error(f"Ошибка при анализе {os.path.basename(image_path)}: {str(e)}")

        progress.close()

        if result_image_paths:
            self.results_window = ResultsWindow(result_image_paths)
            self.results_window.exec()
        else:
            self.show_info("Объекты не обнаружены.")

    def show_error(self, message):
        msg = QMessageBox(self)
        msg.setIcon(QMessageBox.Critical)
        msg.setWindowTitle("Ошибка")
        msg.setText(message)
        msg.setStyleSheet("""
            QMessageBox {
                background-color: #1a1a2e;
            }
            QLabel {
                color: white;
            }
        """)
        msg.exec()

    def show_info(self, message):
        msg = QMessageBox(self)
        msg.setIcon(QMessageBox.Information)
        msg.setWindowTitle("Информация")
        msg.setText(message)
        msg.setStyleSheet("""
            QMessageBox {
                background-color: #1a1a2e;
            }
            QLabel {
                color: white;
            }
        """)
        msg.exec()

class YOLOTrainerApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.selected_model = None
        self.trained_model_path = None
        
        palette = QPalette()
        palette.setColor(QPalette.Window, QColor(26, 26, 46))
        palette.setColor(QPalette.WindowText, QColor(224, 224, 224))
        palette.setColor(QPalette.Base, QColor(40, 40, 60))
        palette.setColor(QPalette.AlternateBase, QColor(50, 50, 70))
        palette.setColor(QPalette.ToolTipBase, QColor(0, 168, 255))
        palette.setColor(QPalette.ToolTipText, QColor(255, 255, 255))
        palette.setColor(QPalette.Text, QColor(224, 224, 224))
        palette.setColor(QPalette.Button, QColor(0, 168, 255))
        palette.setColor(QPalette.ButtonText, QColor(255, 255, 255))
        palette.setColor(QPalette.BrightText, QColor(255, 255, 255))
        palette.setColor(QPalette.Highlight, QColor(0, 168, 255))
        palette.setColor(QPalette.HighlightedText, QColor(0, 0, 0))
        self.setPalette(palette)
        
        self.initUI()

    def initUI(self):
        self.setWindowTitle("Neural Vision Trainer")
        self.setMinimumSize(900, 700)
        
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        
        main_layout = QVBoxLayout()
        main_layout.setContentsMargins(40, 40, 40, 40)
        main_layout.setSpacing(30)
        
        header = QLabel("YOLOv8")
        header.setStyleSheet("""
            QLabel {
                font-size: 36px;
                font-weight: bold;
                color: #00a8ff;
                text-align: center;
                letter-spacing: 2px;
            }
        """)
        header.setAlignment(Qt.AlignCenter)
        main_layout.addWidget(header)
        
        subheader = QLabel("Обнаружение разливов нефти по фото")
        subheader.setStyleSheet("""
            QLabel {
                font-size: 18px;
                color: #a0a0a0;
                text-align: center;
            }
        """)
        subheader.setAlignment(Qt.AlignCenter)
        main_layout.addWidget(subheader)
        
        button_layout = QHBoxLayout()
        button_layout.setSpacing(30)
        
        self.model_training_button = FuturisticButton("Обучение модели")
        self.model_training_button.setIcon(QIcon("icons/train.png"))
        self.model_training_button.clicked.connect(self.open_model_training_window)
        
        self.load_trained_model_button = FuturisticButton("Загрузить модель")
        self.load_trained_model_button.setIcon(QIcon("icons/upload.png"))
        self.load_trained_model_button.clicked.connect(self.load_trained_model)
        
        self.image_analysis_button = FuturisticButton("Анализ изображений")
        self.image_analysis_button.setIcon(QIcon("icons/analyze.png"))
        self.image_analysis_button.clicked.connect(self.open_image_analysis_window)
        
        button_layout.addWidget(self.model_training_button)
        button_layout.addWidget(self.load_trained_model_button)
        button_layout.addWidget(self.image_analysis_button)
        main_layout.addLayout(button_layout)
        
        info_group = QGroupBox("Состояние системы")
        info_group.setStyleSheet("""
            QGroupBox {
                border: 2px solid #00a8ff;
                border-radius: 8px;
                margin-top: 15px;
                padding-top: 20px;
                font-weight: bold;
                color: #00a8ff;
                font-size: 16px;
                background-color: rgba(26, 26, 46, 150);
            }
        """)
        
        info_layout = QVBoxLayout()
        
        self.status_label = QLabel("Статус: система готова")
        self.status_label.setStyleSheet("font-size: 14px; color: #e0e0e0;")
        
        self.model_info_text = QTextEdit()
        self.model_info_text.setReadOnly(True)
        self.model_info_text.setStyleSheet("""
            QTextEdit {
                background-color: rgba(40, 40, 60, 200);
                border: 1px solid #00a8ff;
                border-radius: 6px;
                padding: 15px;
                font-size: 14px;
                color: white;
            }
        """)
        self.model_info_text.setPlaceholderText("Информация о модели появится здесь...")
        
        info_layout.addWidget(self.status_label)
        info_layout.addWidget(self.model_info_text)
        info_group.setLayout(info_layout)
        main_layout.addWidget(info_group)
        
        central_widget.setLayout(main_layout)
        self.setWindowIcon(QIcon("icons/app.png"))

    def open_model_training_window(self):
        self.model_training_window = ModelTrainingWindow(self)
        self.model_training_window.exec()

    def load_trained_model(self):
        model_path, _ = QFileDialog.getOpenFileName(
            self,
            "Выберите файл модели",
            "",
            "Model Files (*.pt)"
        )
        if model_path:
            self.trained_model_path = model_path
            self.selected_model = None
            self.status_label.setText(f"Загружена модель: {os.path.basename(model_path)}")
            
            try:
                model = YOLO(model_path)
                model_info = self.get_model_info(model)
                metrics = self.load_metrics_from_results(model_path)
                
                html = [
                    "<style>td {padding: 2px 10px;}</style>",
                    "<b>Основные параметры:</b><br>",
                    f"<table><tr><td>Название:</td><td>{os.path.basename(model_path)}</td></tr>",
                    f"<tr><td>Размер:</td><td>{os.path.getsize(model_path)/1024/1024:.2f} MB</td></tr>",
                    f"<tr><td>Классов:</td><td>{model_info.get('nc', 'N/A')}</td></tr></table><br>"
                ]

                if metrics:
                    html.extend([
                        "<b>Метрики модели:</b><br>",
                        "<table>",
                        f"<tr><td>Precision:</td><td>{metrics.get('precision', 0):.3f}</td></tr>",
                        f"<tr><td>Recall:</td><td>{metrics.get('recall', 0):.3f}</td></tr>",
                        f"<tr><td>mAP@0.5:</td><td>{metrics.get('map50', 0):.3f}</td></tr>",
                        f"<tr><td>mAP@0.5-0.95:</td><td>{metrics.get('map', 0):.3f}</td></tr>",
                        "</table><br>"
                    ])
                else:
                    html.append("<i>Метрики не найдены (ищите файл results.csv рядом с моделью)</i><br><br>")

                if 'names' in model_info:
                    html.append("<b>Классы модели:</b><br><ul>")
                    for i, name in model_info['names'].items():
                        html.append(f"<li>{i}: {name}</li>")
                    html.append("</ul>")

                self.model_info_text.setHtml(''.join(html))
                
            except Exception as e:
                self.model_info_text.setPlainText(f"Ошибка при загрузке модели: {str(e)}\n\nМодель может быть использована, но метрики недоступны.")

    def get_model_info(self, model):
        info = {}
        if hasattr(model, 'model'):
            if hasattr(model.model, 'names'):
                info['names'] = model.model.names
            if hasattr(model.model, 'nc'):
                info['nc'] = model.model.nc
        return info

    def load_metrics_from_results(self, model_path):
        # Ищем файл results.csv в той же директории
        model_dir = os.path.dirname(model_path)
        results_file = os.path.join(model_dir, "results.csv")
        
        if not os.path.exists(results_file):
            return None
            
        try:
            df = pd.read_csv(results_file)
            last_row = df.iloc[-1]  # Берем последнюю эпоху
            
            return {
                'precision': last_row.get('metrics/precision(B)', 0),
                'recall': last_row.get('metrics/recall(B)', 0),
                'map50': last_row.get('metrics/mAP50(B)', 0),
                'map': last_row.get('metrics/mAP50-95(B)', 0)
            }
        except Exception as e:
            print(f"Ошибка при чтении results.csv: {str(e)}")
            return None

    def open_image_analysis_window(self):
        if not self.selected_model and not self.trained_model_path:
            self.show_error("Сначала выберите или загрузите модель!")
            return
        self.image_analysis_window = ImageAnalysisWindow(self)
        self.image_analysis_window.exec()

    def show_error(self, message):
        msg = QMessageBox(self)
        msg.setIcon(QMessageBox.Critical)
        msg.setWindowTitle("Ошибка")
        msg.setText(message)
        msg.exec()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    app.setStyle("Fusion")
    window = YOLOTrainerApp()
    window.show()
    sys.exit(app.exec())